In [0]:
%run /Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/com_utl_fun

In [0]:
import json

gendral_config = dbutils.notebook.run("/Workspace/Users/loganathansundaramsa@gmail.com/dataengg_inc_learning/databricks_learning/Brazilian_Ecommerce/gendral_config/common_notebook", 120)


config_dict = json.loads(gendral_config)

src = config_dict['src']
bronze = config_dict['bronze']
silver = config_dict['silver']
gold = config_dict['gold']
catalog = config_dict['CATALOG']
schema = config_dict['SCHEMA']

In [0]:
df_geolocation = read_file(spark,"parquet",f'{bronze}/geolocation')

columns =  {"geolocation_zip_code_prefix":"zip_code_prefix",
            "geolocation_lat":"latitude",
            "geolocation_lng":"longitude",
            "geolocation_city":"city",
            "geolocation_state":"state"}

df_std_name = df_geolocation.withColumnsRenamed(columns);


In [0]:
df_dedup = deduplicate(df_std_name,["zip_code_prefix","latitude","longitude"])


columns = {
        "latitude": "DOUBLE",
        "longitude":"DOUBLE"}

df_cast_clms = f_cast_col(df_dedup,columns).filter("latitude is not null and longitude is not null")

df_group = df_cast_clms.groupBy("zip_code_prefix").agg(avg("latitude").alias("avg_latitude"),avg("longitude").alias("avg_longitude"))


df_geoloc_fin = df_cast_clms .join(df_group,on = "zip_code_prefix", how = "inner")

columns = {
        "created_at": {"type": "timestamp"},
        "updated_at": {"type": "timestamp"},
        "source": {"type": "string", "default": "geolocation"}}

df_final_geo = enrich_add_col(df_geoloc_fin, columns)

print(df_dedup.count(),df_cast_clms.count(),df_group.count(),df_geoloc_fin.count())
# df_geoloc_fin.display()


In [0]:
write_file(df_final_geo,f"{silver}/silver_geolocation")

write_delta_liquid(df_final_geo, f"{catalog}.{schema}.silver_geolocation",["zip_code_prefix"])